In [1]:
import math
import numpy as np
from qiskit import *
from qiskit.tools.visualization import plot_histogram, plot_state_city
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator, UnitarySimulator
from qiskit import IBMQ
import matplotlib
%matplotlib inline

IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [2]:
def getStatevectorFromImage(path):
    from PIL import Image
    image= Image.open(path)
    (width, height) = image.size
    pixels = image.load()

    ssum = 0
    for i in range(width):
        for j in range(height):
            ssum += pixels[i,j]**2

    m = math.floor(math.log(height, 2))
    n = math.floor(math.log(width, 2))
    stateVector = np.zeros(2**(m+n))
    for i in range(width):
        for j in range(height):
            stateVector[i*height + j] = pixels[i,j]/math.sqrt(ssum)
            
    return stateVector, m, n

In [3]:
sVector = getStatevectorFromImage("mlk.jpg")[0]
print("sVector: ",sum(sVector**2))

sVector:  1.0000000000000098


In [4]:
def Translate(increment, quantumRegister, circuit):
    from qiskit.aqua.components.qfts import Standard as qft
    from qiskit.aqua.components.iqfts import Standard as iqft
    
    n = len(quantumRegister)
    
    qft(n).construct_circuit(qubits=quantumRegister,circuit=circuit)
    
    for j in range(n):
        circuit.u1((np.pi*increment)/(2**(n-1-j)), quantumRegister[j])
        
    iqft(n).construct_circuit(qubits=quantumRegister,circuit=circuit)

In [5]:
def exportQuantumImage(counts, shots, height, width):
    from PIL import Image

    r = math.floor(math.log(width, 2))
    img= Image.new("L", (width, height))
    pixels = img.load()

    maxAmplitude = 256
    medAmplitude = maxAmplitude/2
    med = shots/(height * width)
    for key in counts:
        i = int(key[0:r], 2)
        j = int(key[r:], 2)
        val = round((((counts[key] - med)/med)*medAmplitude) + medAmplitude)
        pixels[i,j] = (val)
        
    return img

In [ ]:
from qiskit.aqua.components.qfts import Standard as qft
stateVector, m, n = getStatevectorFromImage('mlk.jpg')

state = [complex(x) for x in stateVector]
indexes = range(m+n)

columnReg = QuantumRegister(n)
rowReg = QuantumRegister(m)
creg = ClassicalRegister(n+m)

circ = QuantumCircuit(rowReg, columnReg, creg)
circ.initialize(state, indexes)

#qft(m).construct_circuit(qubits=rowReg,circuit=circ)
#qft(n).construct_circuit(qubits=columnReg,circuit=circ)

#Translate(2**(n-1), rowReg, circ)
#Translate(2**(m-1), columnReg, circ)
circ.measure(indexes, indexes)

provider = IBMQ.get_provider(group='open')
backend = provider.get_backend('ibmq_qasm_simulator')

shots = 2024
result = execute(circ, backend, shots=shots).result()
counts = result.get_counts(circ)
#plot_histogram(counts)

In [6]:
exportQuantumImage(counts,shots,2**m,2**n)

NameError: name 'counts' is not defined